<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# **SQL Server 2019 big data cluster Tutorial**
## **03 - SQL Server Master Instance Queries**

In this tutorial you will learn how to run standard SQL Server Queries against the Master Instance (MI) in a SQL Server big data cluster. 

You'll start with a simple set of queries to explore the Instance: 


In [ ]:
/* Instance Version */
SELECT @@VERSION; 
GO

/* General Configuration */
USE master;  
GO  
EXEC sp_configure;
GO

/* Databases on this Instance */
SELECT db.name AS 'Database Name'
, Physical_Name AS 'Location on Disk'
, Cast(Cast(Round(cast(mf.size as decimal) * 8.0/1024000.0,2) as decimal(18,2)) as nvarchar) 'Size (GB)'
FROM sys.master_files mf
INNER JOIN 
    sys.databases db ON db.database_id = mf.database_id
WHERE mf.type_desc = 'ROWS';
GO

SELECT * from sys.master_files


## **Ingest data into the SQL Server Databases**

Before you start working with data, you need to briloadng it in to the system. 

You have several options to do that, from the `bcp` utility to SQL Server Integration Services, the Azure Data Factory and more. 

For the structured data, you'll use the SQL Server `RESTORE` command to bring in two databases from the location we specified earlier with the `kubectl` command.

The Code below shows all of that: 

In [ ]:
/* Add the Customer Databases to the cluster */
USE [master]
RESTORE DATABASE [analysis] FROM  DISK = N'/var/opt/mssql/data/analysis.bak' 
WITH  FILE = 1
,  MOVE N'tpcxbb_1gb' TO N'/var/opt/mssql/data/analysis.mdf'
,  MOVE N'tpcxbb_1gb_log' TO N'/var/opt/mssql/data/analysis.ldf'
,  NOUNLOAD
,  REPLACE
,  STATS = 5
GO

USE [master]
RESTORE DATABASE [sales] FROM  DISK = N'/var/opt/mssql/data/sales.bak' 
WITH  FILE = 1
,  MOVE N'tpcxbb_1gb' TO N'/var/opt/mssql/data/sales.mdf'
,  MOVE N'tpcxbb_1gb_log' TO N'/var/opt/mssql/data/sales.ldf'
,  NOUNLOAD
,  REPLACE
,  STATS = 5
GO

USE [master]
RESTORE DATABASE [WideWorldImporters] FROM  DISK = N'/var/opt/mssql/data/WWI.bak' WITH  FILE = 1
,  MOVE N'WWI_Primary' TO N'/var/opt/mssql/data/WideWorldImporters.mdf'
,  MOVE N'WWI_UserData' TO N'/var/opt/mssql/data/WideWorldImporters_UserData.ndf'
,  MOVE N'WWI_Log' TO N'/var/opt/mssql/data/WideWorldImporters.ldf'
,  MOVE N'WWI_InMemory_Data_1' TO N'/var/opt/mssql/data/WideWorldImporters_InMemory_Data_1'
,  NOUNLOAD
,  REPLACE
,  STATS = 5
GO

USE [master]
RESTORE DATABASE [AdventureWorks] FROM  DISK = N'/var/opt/mssql/data/AdventureWorks.bak' 
WITH  FILE = 1
,  MOVE N'AdventureWorks2017' TO N'/var/opt/mssql/data/AdventureWorks2017.mdf'
,  MOVE N'AdventureWorks2017_log' TO N'/var/opt/mssql/data/AdventureWorks2017_log.ldf'
,  NOUNLOAD
,  REPLACE
,  STATS = 5
GO

USE [master]
RESTORE DATABASE [AdventureWorksDW] FROM  DISK = N'/var/opt/mssql/data/AdventureWorksDW.bak' 
WITH  FILE = 1,  MOVE N'AdventureWorksDW2017' TO N'/var/opt/mssql/data/AdventureWorksDW.mdf'
,  MOVE N'AdventureWorksDW2017_log' TO N'/var/opt/mssql/data/AdventureWorksDW_log.ldf'
,  NOUNLOAD
,  REPLACE
,  STATS = 5
GO

USE [master]
RESTORE DATABASE [NYC] FROM  DISK = N'/var/opt/mssql/data/NYC.bak' 
WITH  FILE = 1
,  MOVE N'NYCTaxi_Sample' TO N'/var/opt/mssql/data/NYC.mdf'
,  MOVE N'NYCTaxi_Sample_log' TO N'/var/opt/mssql/data/NYC_0.ldf'
,  NOUNLOAD
,  REPLACE
,  STATS = 5
GO


## **Query Data**

The SQL Server 2019 big data cluster Master Instance is a SQL Server Instance - and as such has most all of the query facilities and capabilities of Microsoft SQL Server running on Linux.

In these queries you'll explore some customer information, in preparation for combining that information with external sources. You can add cells to this workbook to explore more. 

In [ ]:
USE WideWorldImporters;
SELECT * FROM INFORMATION_SCHEMA.tables ORDER BY TABLE_SCHEMA

In [ ]:
/* Get some general information about the data in the WWI OLTP system */

/* Show the Populations. 
Where do we have the most people?
 */
SELECT TOP 10 CityName as 'City Name'
, StateProvinceName as 'State or Province'
, sp.LatestRecordedPopulation as 'Population'
, CountryName
FROM Application.Cities AS city
JOIN Application.StateProvinces AS sp ON
    city.StateProvinceID = sp.StateProvinceID
JOIN Application.Countries AS ctry ON 
    sp.CountryID=ctry.CountryID
ORDER BY Population, CityName;
GO


In [ ]:
/* Show Customer Sales in WWI OLTP */
USE WideWorldImporters;
GO

SELECT TOP 10 s.CustomerID
, s.CustomerName
, sc.CustomerCategoryName
,  pp.FullName AS PrimaryContact
,  ap.FullName AS AlternateContact
,  s.PhoneNumber
,  s.FaxNumber
,  bg.BuyingGroupName
,  s.WebsiteURL
,  dm.DeliveryMethodName AS DeliveryMethod
,  c.CityName AS CityName
,  s.DeliveryLocation AS DeliveryLocation
,  s.DeliveryRun
,  s.RunPosition
FROM Sales.Customers AS s
    LEFT OUTER JOIN Sales.CustomerCategories AS sc
    ON s.CustomerCategoryID = sc.CustomerCategoryID
    LEFT OUTER JOIN [Application].People AS pp
    ON s.PrimaryContactPersonID = pp.PersonID
    LEFT OUTER JOIN [Application].People AS ap
    ON s.AlternateContactPersonID = ap.PersonID
    LEFT OUTER JOIN Sales.BuyingGroups AS bg
    ON s.BuyingGroupID = bg.BuyingGroupID
    LEFT OUTER JOIN [Application].DeliveryMethods AS dm
    ON s.DeliveryMethodID = dm.DeliveryMethodID
    LEFT OUTER JOIN [Application].Cities AS c
    ON s.DeliveryCityID = c.CityID
ORDER BY c.CityName

## **Next Step: Data Virtualization**

Now you're ready to open the next Jupyter Notebook - `bdc_tutorial_04.ipynb` - to learn how to work with Data Virtualization.